<a href="https://colab.research.google.com/github/mohit3agarwal/kungfu-A3C/blob/main/Kung_Fu_A3C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kung Fu using A3C Algorithm

---

## Installing the required packages and importing the libraries

### Installing Gymnasium

In [1]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.0 MB/s eta 0:00:00
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446662 sha256=bf9ef3fda823574fa2e3c5a2e30deafc717165a0014af286743eb29084517007
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed

### Importing the libraries

In [2]:
import cv2
import math
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.multiprocessing as mp
import torch.distributions as distributions
from torch.distributions import Categorical
import gymnasium as gym
from gymnasium import ObservationWrapper
from gymnasium.spaces import Box

## Building the AI

### Creating the architecture of the Neural Network

In [3]:
class Network(nn.Module):

  def __init__(self, action_size):
    super(Network, self).__init__()                                                                       # Activate inheritance and initialize the parent class
    self.conv1 = torch.nn.Conv2d(in_channels = 4,  out_channels = 32, kernel_size = (3,3), stride = 2)    # setting the convolution layers
    self.conv2 = torch.nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = (3,3), stride = 2)
    self.conv3 = torch.nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = (3,3), stride = 2)
    self.flatten = torch.nn.Flatten()                                                                     # Flatten the output of the last convolution layer
    self.fc1  = torch.nn.Linear(512, 128)                                                                 # 1st fully connected layer
    self.fc2a = torch.nn.Linear(128, action_size)                                                         # output layer for action values
    self.fc2s = torch.nn.Linear(128, 1)                                                                   # output layer for state value

  # defining the forward propagation method and using ReLU activation
  def forward(self, state):
    x = self.conv1(state)
    x = F.relu(x)
    x = self.conv2(x)
    x = F.relu(x)
    x = self.conv3(x)
    x = F.relu(x)
    x = self.flatten(x)
    x = self.fc1(x)
    x = F.relu(x)

    action_values = self.fc2a(x)       # forward propagating to action values layer
    state_value = self.fc2s(x)[0]      # forward propagating to state value layer

    return action_values, state_value

## Training the AI

### Setting up the environment

In [7]:
class PreprocessAtari(ObservationWrapper):

  def __init__(self, env, height = 42, width = 42, crop = lambda img: img, dim_order = 'pytorch', color = False, n_frames = 4):
    super(PreprocessAtari, self).__init__(env)
    self.img_size = (height, width)
    self.crop = crop
    self.dim_order = dim_order
    self.color = color
    self.frame_stack = n_frames
    n_channels = 3 * n_frames if color else n_frames
    obs_shape = {'tensorflow': (height, width, n_channels), 'pytorch': (n_channels, height, width)}[dim_order]
    self.observation_space = Box(0.0, 1.0, obs_shape)
    self.frames = np.zeros(obs_shape, dtype = np.float32)

  def reset(self):
    self.frames = np.zeros_like(self.frames)
    obs, info = self.env.reset()
    self.update_buffer(obs)
    return self.frames, info

  def observation(self, img):
    img = self.crop(img)
    img = cv2.resize(img, self.img_size)
    if not self.color:
      if len(img.shape) == 3 and img.shape[2] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = img.astype('float32') / 255.
    if self.color:
      self.frames = np.roll(self.frames, shift = -3, axis = 0)
    else:
      self.frames = np.roll(self.frames, shift = -1, axis = 0)
    if self.color:
      self.frames[-3:] = img
    else:
      self.frames[-1] = img
    return self.frames

  def update_buffer(self, obs):
    self.frames = self.observation(obs)

def make_env():
  env = gym.make("KungFuMasterDeterministic-v0", render_mode = 'rgb_array');
  env = PreprocessAtari(env, height = 42, width = 42, crop = lambda img: img, dim_order = 'pytorch', color = False, n_frames = 4)
  return env

env = make_env();

state_shape = env.observation_space.shape
number_actions = env.action_space.n
print("State shape:", state_shape)
print("Number actions:", number_actions)
print("Action names:", env.env.env.get_action_meanings())

State shape: (4, 42, 42)
Number actions: 14
Action names: ['NOOP', 'UP', 'RIGHT', 'LEFT', 'DOWN', 'DOWNRIGHT', 'DOWNLEFT', 'RIGHTFIRE', 'LEFTFIRE', 'DOWNFIRE', 'UPRIGHTFIRE', 'UPLEFTFIRE', 'DOWNRIGHTFIRE', 'DOWNLEFTFIRE']


### Initializing the hyperparameters

In [8]:
learning_rate = 1e-4        # setting the learning rate
discount_factor = 0.99      # setting the discount factor (aka gamma)
number_environments = 10    # setting the number of environments (or agents) to train on

### Implementing the A3C class

In [9]:
class Agent():

  def __init__(self, action_size):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.action_size = action_size
    self.network = Network(action_size).to(self.device)
    self.optimizer = torch.optim.Adam(self.network.parameters(), lr = learning_rate)   # using the Adam class for optimizer

  def act(self, state):
    if state.ndim == 3:    # checking if state is in a batch, if its dimension is 3D convert to 4D
      state = [state]

    state = torch.tensor(state, dtype = torch.float32, device = self.device)      # convert state to pytorch tensor
    action_values, _ = self.network(state)                                        # forward propagation to get action values

    policy = F.softmax(action_values, dim = -1)                                   # implementing softmax policy

    return np.array([np.random.choice(len(p), p = p) for p in policy.detach().cpu().numpy()])  # return sample action from policy

  # implementing the A3C algorithm
  def step(self, state, action, reward, next_state, done):
    batch_size = state.shape[0]

    # convert to pytorch tensor
    state = torch.tensor(state, dtype = torch.float32, device = self.device)
    next_state = torch.tensor(next_state, dtype = torch.float32, device = self.device)
    reward = torch.tensor(reward, dtype = torch.float32, device = self.device)
    done = torch.tensor(done, dtype = torch.bool, device = self.device).to(dtype = torch.float32)

    action_values, state_value = self.network(state)                                   # forward propagation to get action values and state value
    _, next_state_value = self.network(next_state)                                     # forward propagation to get next state value

    target_state_value = reward + discount_factor * next_state_value * (1 - done)      # calculate target state value using the bellman eqn
    advantage = target_state_value - state_value                                       # calculate advantage

    # calculate entropy
    probs = F.softmax(action_values, dim = -1)
    logprobs = F.log_softmax(action_values, dim = -1)
    entropy = -torch.sum(probs * logprobs, axis = -1)

    # calculate log probability of actions taken
    batch_idx = np.arange(batch_size)
    logp_actions = logprobs[batch_idx, action]

    # calculate actor-critic losses
    actor_loss = -(logp_actions * advantage.detach()).mean() - 0.001 * entropy.mean()
    critic_loss = F.mse_loss(target_state_value.detach(), state_value)
    total_loss = actor_loss + critic_loss

    # update the network parameters
    self.optimizer.zero_grad()
    total_loss.backward()
    self.optimizer.step()

### Initializing the A3C agent

In [10]:
agent = Agent(number_actions)

### Evaluating our A3C agent on a certain number of episodes

In [11]:
def evaluate(agent, env, n_episodes = 1):
  episodes_rewards = []
  for _ in range(n_episodes):
    state, _ = env.reset()      # initialise the state
    total_reward = 0
    while True:
      action = agent.act(state)                           # take an action
      state, reward, done, info, _ = env.step(action[0])  # execute the action
      total_reward += reward                              # add reward
      if done:
        break
    episodes_rewards.append(total_reward)
  return episodes_rewards

### Managing multiple environments simultaneously

In [12]:
class EnvBatch:

  def __init__(self, n_envs = 10):
    self.envs = [make_env() for _ in range(n_envs)]    # initialise the environments

  def reset(self):
    _states = []
    for env in self.envs:
      _states.append(env.reset()[0])     # append the initialised states
    return np.array(_states)

  def step(self, actions):
    next_states, rewards, dones, infos, _ = map(np.array, zip(*[env.step(a) for env, a in zip(self.envs, actions)]))  # execute the actions in each env

    # check if any env is done
    for i in range(len(self.envs)):
      if dones[i]:
        next_states[i] = self.envs[i].reset()[0]    # reset the env if it is done

    return next_states, rewards, dones, infos

### Training the A3C agent

In [13]:
import tqdm   # to display progress bars of loops

env_batch = EnvBatch(number_environments)     # initialise the number of environments
batch_states = env_batch.reset();             # initialise and reset the batch states across all environments

with tqdm.trange(0, 3001) as progress_bar:
  for i in progress_bar:
    batch_actions = agent.act(batch_states)                                                  # take an action from all the agents
    batch_next_states, batch_rewards, batch_dones, _ = env_batch.step(batch_actions)         # execute the action in the environments
    batch_rewards *= 0.01                                                                    # scale down the rewards to stabilise the training
    agent.step(batch_states, batch_actions, batch_rewards, batch_next_states, batch_dones)   # update the agent
    batch_states = batch_next_states                                                         # update the batch states

    if i % 1000 == 0:
      print("Average agent reward: ", np.mean(evaluate(agent, env, n_episodes = 10)))

/usr/local/lib/python3.10/dist-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment KungFuMasterDeterministic-v0 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(
  0%|          | 0/3001 [00:00<?, ?it/s]<ipython-input-9-3b966d1356f9>:47: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  critic_loss = F.mse_loss(target_state_value.detach(), state_value)
<ipython-input-9-3b966d1356f9>:13: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  state = torch.tensor(state, dtype = torch.float32, device = self.device)      # convert state to pytorch tensor
  0%|  

Average agent reward:  580.0


 34%|███▎      | 1006/3001 [01:48<1:12:52,  2.19s/it]

Average agent reward:  830.0


 67%|██████▋   | 2004/3001 [03:01<44:15,  2.66s/it]

Average agent reward:  1180.0


100%|██████████| 3001/3001 [04:10<00:00, 12.00it/s]

Average agent reward:  990.0


## Visualizing the results

In [17]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gymnasium.wrappers.monitoring.video_recorder import VideoRecorder

def show_video_of_model(agent, env):
  state, _ = env.reset()
  done = False
  frames = []
  while not done:
    frame = env.render()
    frames.append(frame)
    action = agent.act(state)
    state, reward, done, _, _ = env.step(action[0])
  env.close()
  imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, env)

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()